## Бейзлайны и детерминированные алгоритмы item-item

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(5)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    recs = np.random.choice(items_weights['item_id'].values.ravel(), 5, replace=False, p=items_weights['weight'].values.ravel())
    
    return recs.tolist()

In [6]:
%%time
item_id = data['item_id'].unique()
weights = np.random.dirichlet(np.ones(item_id.shape[0]), size=1)
df = pd.DataFrame({'item_id': item_id, 'weight': weights.ravel()})
recs = weighted_random_recommendation(df)
recs

Wall time: 38 ms


[9707376, 1111162, 899073, 1091472, 1794655]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [8]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [9]:
actual = data_test.groupby('user_id')['item_id'].unique().reset_index()
actual.head()

,user_id,item_id
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [10]:
result.drop(columns='actual', axis=1, inplace=True)

In [11]:
result = result.merge(actual, how='left', on='user_id')
result.rename(columns={'item_id': 'actual'}, inplace=True)
result.head()

,user_id,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,actual
0,1,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]","[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[5574336, 990072, 868548, 995880, 842226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]","[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[1277401, 94446, 3133282, 1925252, 855699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]","[835098, 872137, 910439, 924610, 992977, 10412..."


In [19]:
def str_to_list(string):
    if isinstance(string, list) or isinstance(string, int) or isinstance(string, np.ndarray):
        return string
    elif isinstance(string, str):
        result = []
        splitted_list = string[1:-1].split(', ')
        for value in splitted_list:
            try:
                result.append(int(value))
            except ValueError:
                print(f'Value {value} not added')
        return result
    else:
        raise ValueError('Wrong value passed')

In [20]:
for column in result.columns:
    result[column] = result[column].map(str_to_list)

In [21]:
def precision_at_k_(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [22]:
def precision_at_k_for_df(df, k=5):
    columns = [x for x in df.columns if x not in ['actual', 'user_id']]
    result = []
    for column in columns:
        column_precisions = []
        for rec_list, bought_list in zip(df[column], df['actual']):
            column_precisions.append(precision_at_k(rec_list, bought_list))
        result.append(np.mean(column_precisions))

    result = dict(zip(columns, result))
    return result

In [23]:
precision_at_k_for_df(result)

{'random_recommendation': 0.0005876591576885408,
 'popular_recommendation': 0.15523996082272282,
 'itemitem': 0.03359451518119491,
 'cosine': 0.03525954946131244,
 'tfidf': 0.036141038197845254,
 'own_purchases': 0.17998694090760692}

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [24]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)  # n_sold - число продаж

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [25]:
data_train2 = data_train.loc[data_train['item_id'].isin(top_5000), :]

In [26]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']

In [27]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""

    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)

    return recs.tolist()

In [28]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""

    popular = data.groupby(by='item_id')['sales_value'].sum().reset_index()
    popular.sort_values(by='sales_value', ascending=False, inplace=True)

    recs = popular.head(n).item_id

    return recs.tolist()

In [29]:
%%time
items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

Wall time: 3.27 s


In [31]:
%%time
items = data_train2.item_id.unique()
result['random_recommendation_2'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

Wall time: 239 ms


In [32]:
%%time
popular_recs = popularity_recommendation(data_train, n=5)
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

Wall time: 105 ms


In [33]:
%%time
popular_recs = popularity_recommendation(data_train2, n=5)
result['popular_recommendation_2'] = result['user_id'].apply(lambda x: popular_recs)

Wall time: 46 ms


In [34]:
result.head(2).T

,0,1
user_id,1,3
actual,"[821867, 834484, 856942, 865456, 889248, 90795...","[835476, 851057, 872021, 878302, 879948, 90963..."
random_recommendation,"[858897, 10311277, 12427953, 869282, 7166813]","[1446177, 1216756, 7441339, 912311, 926900]"
random_recommendation_2,"[993466, 944486, 990688, 837012, 915617]","[6979427, 996028, 850679, 1081355, 6424472]"
popular_recommendation,"[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]"
popular_recommendation_2,"[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [35]:

pd.DataFrame([
    result.apply(lambda row: precision_at_k_(row['random_recommendation'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k_(row['random_recommendation_2'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k_(row['popular_recommendation'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k_(row['popular_recommendation_2'], row['actual']), axis=1).mean(),
], 
    columns=['precision_at_k'], 
    index=['random_recommendation', 'random_recommendation_5000', 'popular_recommendation', 'popular_recommendation_5000']
).T

,random_recommendation,random_recommendation_5000,popular_recommendation,popular_recommendation_5000
precision_at_k,0.000294,0.00617,0.15524,0.15524


In [36]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']

In [37]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id',
                                  columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0)

user_item_matrix[user_item_matrix > 0] = 1  # так как в итоге хотим предсказать
user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
# user_item_matrix.head(3)

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [42]:
%%time

for k in range(5, 11):
    
    model = ItemItemRecommender(K=k, num_threads=4)  # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)
    
    recs = model.recommend(
        userid=userid_to_id[2],  # userid - id от 0 до N
        user_items=csr_matrix(user_item_matrix).tocsr(),  # на вход user-item matrix
        N=5,  # кол-во рекомендаций 
        filter_already_liked_items=False,
        filter_items=None,
        recalculate_user=True)
    
    result[f'itemitem_K={k}'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 4.03 s


In [43]:
pd.DataFrame(
    [
        result.apply(lambda row: precision_at_k(row['itemitem_K=5'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=6'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=7'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=8'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=9'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=10'], row['actual']), axis=1).mean(),
    ], 
    columns = ['precision_at_k'], index=[f'k={i}' for i in range(5, 11)]
).sort_values(by='precision_at_k', ascending=False).T

,k=10,k=9,k=8,k=7,k=6,k=5
precision_at_k,0.15093,0.148482,0.147209,0.144858,0.142018,0.136925
